# Context-Free Grammars and Pushdown Automata

This notebook explores context-free languages through grammars and pushdown automata.

## Learning Objectives
- Understand context-free grammars (CFGs)
- Master pushdown automaton construction
- Explore stack-based computation
- Recognize nested language structures

In [ ]:
import sys
sys.path.append('../src')
from automata import PDA

## 1. Simple Context-Free Language: Balanced Parentheses

The classic example of a context-free language that's not regular.

In [ ]:
# PDA for balanced parentheses
balanced_parens = PDA(
    states={'q0', 'q1'},
    alphabet={'(', ')'},
    stack_alphabet={'Z', 'P'},
    transitions={
        ('q0', '(', 'Z'): {('q0', 'PZ')},   # Push P onto empty stack
        ('q0', '(', 'P'): {('q0', 'PP')},   # Push P onto P
        ('q0', ')', 'P'): {('q0', '')},     # Pop P for ')'
        ('q0', '', 'Z'): {('q1', 'Z')}      # Accept when stack has only Z
    },
    start_state='q0',
    accept_states={'q1'}
)

# Test balanced parentheses
test_cases = [
    ('', True),
    ('()', True),
    ('(())', True),
    ('()()', True),
    ('((()))', True),
    ('(', False),
    (')', False),
    ('(()', False),
    ('())', False),
    ('))(', False)
]

print("Testing PDA for balanced parentheses:")
for string, expected in test_cases:
    result = balanced_parens.accepts(string)
    status = "✓" if result == expected else "✗"
    print(f"'{string}': {'ACCEPT' if result else 'REJECT'} {status}")

## 2. Language a^n b^n

Another classic context-free language: equal numbers of a's followed by b's.

In [ ]:
# PDA for a^n b^n (n >= 0)
a_n_b_n = PDA(
    states={'q0', 'q1', 'q2'},
    alphabet={'a', 'b'},
    stack_alphabet={'Z', 'A'},
    transitions={
        ('q0', 'a', 'Z'): {('q0', 'AZ')},   # First 'a': push A
        ('q0', 'a', 'A'): {('q0', 'AA')},   # More 'a's: push A
        ('q0', 'b', 'A'): {('q1', '')},     # First 'b': pop A, go to q1
        ('q1', 'b', 'A'): {('q1', '')},     # More 'b's: pop A
        ('q1', '', 'Z'): {('q2', 'Z')},     # Accept when stack empty
        ('q0', '', 'Z'): {('q2', 'Z')}      # Accept empty string
    },
    start_state='q0',
    accept_states={'q2'}
)

# Test a^n b^n
test_cases = [
    ('', True),
    ('ab', True),
    ('aabb', True),
    ('aaabbb', True),
    ('aaaabbbb', True),
    ('a', False),
    ('b', False),
    ('aab', False),
    ('abb', False),
    ('abab', False),
    ('ba', False)
]

print("\nTesting PDA for a^n b^n:")
for string, expected in test_cases:
    result = a_n_b_n.accepts(string)
    status = "✓" if result == expected else "✗"
    print(f"'{string}': {'ACCEPT' if result else 'REJECT'} {status}")

## 3. Palindromes with Center Marker

PDA for palindromes of the form wcw^R where w ∈ {a,b}*

In [ ]:
# PDA for palindromes wcw^R
palindrome_pda = PDA(
    states={'q0', 'q1', 'q2'},
    alphabet={'a', 'b', 'c'},
    stack_alphabet={'Z', 'A', 'B'},
    transitions={
        # Push phase: read w and push onto stack
        ('q0', 'a', 'Z'): {('q0', 'AZ')},
        ('q0', 'a', 'A'): {('q0', 'AA')},
        ('q0', 'a', 'B'): {('q0', 'AB')},
        ('q0', 'b', 'Z'): {('q0', 'BZ')},
        ('q0', 'b', 'A'): {('q0', 'BA')},
        ('q0', 'b', 'B'): {('q0', 'BB')},
        
        # Center marker: transition to pop phase
        ('q0', 'c', 'Z'): {('q1', 'Z')},
        ('q0', 'c', 'A'): {('q1', 'A')},
        ('q0', 'c', 'B'): {('q1', 'B')},
        
        # Pop phase: match w^R
        ('q1', 'a', 'A'): {('q1', '')},
        ('q1', 'b', 'B'): {('q1', '')},
        ('q1', '', 'Z'): {('q2', 'Z')}
    },
    start_state='q0',
    accept_states={'q2'}
)

# Test palindromes
test_cases = [
    ('c', True),
    ('aca', True),
    ('bcb', True),
    ('abcba', True),
    ('aabcbaa', True),
    ('abc', False),
    ('abcab', False)
]

print('Testing PDA for palindromes wcw^R:')
for string, expected in test_cases:
    result = palindrome_pda.accepts(string)
    status = '✓' if result == expected else '✗'
    print(f"'{string}': {'ACCEPT' if result else 'REJECT'} {status}")

## 4. Context-Free Grammar Concepts

Understanding the relationship between CFGs and PDAs.

In [ ]:
# Simulate CFG derivation for balanced parentheses
# Grammar: S -> ε | (S) | SS

def generate_balanced_parens(n):
    """Generate all balanced parentheses strings of length 2n"""
    if n == 0:
        return ['']
    
    result = []
    for i in range(n):
        # S -> (S1)S2 where |S1| = 2i, |S2| = 2(n-1-i)
        left_strings = generate_balanced_parens(i)
        right_strings = generate_balanced_parens(n-1-i)
        
        for left in left_strings:
            for right in right_strings:
                result.append('(' + left + ')' + right)
    
    return result

# Generate and test small examples
print('CFG Generation of Balanced Parentheses:')
for n in range(4):
    strings = generate_balanced_parens(n)
    print(f'n={n} (length {2*n}): {strings}')
    
    # Verify with PDA
    for s in strings:
        if not balanced_parens.accepts(s):
            print(f'ERROR: PDA rejected {s}')
            
print('\nAll generated strings accepted by PDA ✓')

## 5. Conclusion

### Key Concepts Learned:

1. **Pushdown Automata**: Stack-based computation for context-free languages
2. **Stack Operations**: Push, pop, and epsilon transitions
3. **Non-determinism**: Multiple transition paths in PDAs
4. **Context-Free Languages**: Beyond regular language capabilities
5. **CFG Equivalence**: Grammars and PDAs recognize the same languages

### Applications:
- Programming language parsing
- Compiler design
- XML/HTML validation
- Mathematical expression evaluation

### Next Steps:
- Explore Turing machines and undecidability
- Study context-sensitive languages
- Learn about parsing algorithms
- Investigate compiler construction techniques